In [ ]:
pip install PyMuPDF
pip install openpyxl
pip install spacy
pip install nltk
pip install scikit-learn
pip install summa

In [ ]:
python -m spacy download en_core_web_sm

In [ ]:
import os
import re
import fitz
import datetime
from openpyxl import Workbook
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from summa.summarizer import summarize as textrank

if not spacy.util.is_package("en_core_web_sm"):
    spacy.cli.download("en_core_web_sm")

nlp = spacy.load("en_core_web_sm")

indian_college_names = [
    "Indian Institute of Technology, Madras",
    "Indian Institute of Technology, Delhi",
    "Indian Institute of Technology, Bombay",
    "Indian Institute of Technology, Kanpur",
    "Indian Institute of Technology, Roorkee",
    "Indian Institute of Technology, Kharagpur",
    "Indian Institute of Technology, Guwahati",
    "Indian Institute of Technology, Hyderabad",
    "Indian Institute of Technology, BHU",
    "Indian Institute of Technology, Dhanbad",
    "Indian Institute of Technology, Indore",
    "Indian Institute of Technology, Ropar",
    "Indian Institute of Technology, Mandi",
    "Indian Institute of Technology, Gandhinagar",
    "Indian Institute of Technology, Jodhpur",
    "Indian Institute of Technology, Patna",
    "Indian Institute of Technology, Bhubaneshwar",
    "Indian Institute of Technology, Tirupati",
    "Indian Institute of Technology, Palakkad",
    "Indian Institute of Technology, Jammu",
    "Indian Institute of Technology, Dharwad",
    "Indian Institute of Technology, Bhilai",
    "National Institute of Technology, Trichy",
    "National Institute of Technology, Karnataka",
    "National Institute of Technology, Rourkela",
    "National Institute of Technology, Warangal",
    "National Institute of Technology, Calicut",
    "National Institute of Technology, Nagpur",
    "National Institute of Technology, Durgapur",
    "National Institute of Technology, Silchar",
    "National Institute of Technology, Jaipur",
    "National Institute of Technology, Allahabad",
    "National Institute of Technology, Kurukshetra",
    "National Institute of Technology, Jalandhar",
    "National Institute of Technology, Surat",
    "National Institute of Technology, Meghalaya",
    "National Institute of Technology, Patna",
    "National Institute of Technology, Raipur",
    "National Institute of Technology, Srinagar",
    "National Institute of Technology, Bhopal",
    "National Institute of Technology, Agartala",
    "National Institute of Technology, Goa",
    "National Institute of Technology, Jamshedpur",
    "National Institute of Technology, Manipur",
    "National Institute of Technology, Hamirpur",
    "National Institute of Technology, Uttarakhand",
    "National Institute of Technology, Puducherry",
    "National Institute of Technology, Arunachal Pradesh",
    "National Institute of Technology, Sikkim",
    "National Institute of Technology, Delhi",
    "National Institute of Technology, Mizoram",
    "National Institute of Technology, Nagaland",
    "National Institute of Technology, Andhra Pradesh",
    "Atal Bihari Vajpayee Indian Institute of Information Technology and Management",
    "Indian Institute of Information Technology, Allahabad",
    "Indian Institute of Information Technology, Design and Manufacturing, Jabalpur",
    "Indian Institute of Information Technology, Design and Manufacturing, Kancheepuram",
    "Indian Institute of Information Technology, Sri City",
    "Indian Institute of Information Technology, Guwahati",
    "Indian Institute of Information Technology, Vadodara",
    "Indian Institute of Information Technology, Kota",
    "Indian Institute of Information Technology, Tiruchirappalli",
    "Indian Institute of Information Technology, Una",
    "Indian Institute of Information Technology, Sonepat",
    "Indian Institute of Information Technology, Kalyani",
    "Indian Institute of Information Technology, Lucknow",
    "Indian Institute of Information Technology, Dharwad",
    "Indian Institute of Information Technology, Design and Manufacturing, Kurnool",
    "Indian Institute of Information Technology, Kottayam",
    "Indian Institute of Information Technology, Manipur",
    "Indian Institute of Information Technology, Nagpur",
    "Indian Institute of Information Technology, Pune",
    "Indian Institute of Information Technology, Ranchi",
    "Indian Institute of Information Technology, Surat",
    "Indian Institute of Information Technology, Bhopal",
    "Indian Institute of Information Technology, Bhagalpur",
    "Indian Institute of Information Technology, Agartala",
    "Indian Institute of Information Technology, Raichur",
    "Smt Bhagwati Chaturvedi College of Engineering, Nagpur",
    "Synergem Power Plant Institute, Nagpur",
    "Nabajyoti College, Kalgachia",
    "National Institute of Technology NIT, Calicut",
    "National Academy of Agricultural Research Management NAARM, Hyderabad",
    "Swami Devi Dyal college of Law, Panchkula",
    "Shree Satguru Dev College of Education, Panchkula",
    "Swami Devi Dayal Hospital & Dental College, Panchkula",
    "Swami Devi Dyal College of Education, Panchkula",
    "Swami Devi Dyal College of Nursing, Panchkula",
    "Jawaharlal Institute of PG Medical Education and Research JIPMER, Puducherry",
    "Sri Guru Gobind Singh College Of Commerce, Delhi",
    "Mahatma Gandhi Institute of Medical Sciences,Wardha",
    "Sri Venkateswara College, Delhi",
    "Kalyani Ray Mahavidyalaya, Rourkela",
    "Galaxy Institute Of Management, Chennai",
    "Jaypee University Of Information Technology, Solan",
    "International Institute of Information Technology IIIT, Hyderabad",
    "University Business School",
    "Mohari Devi Madhav PG College, Sikar",
    "Avinashilingam Institute for Home Science and Higher Education for Women",
    "Narsee Monjee Institute of Management Studies NMIMS, Hyderabad",
    "MGM Medical College & Hospital, Aurangabad",
    "JNTUK University College of Engineering, Vizianagaram",
    "Kongunadu College Of Nursing, Coimbatore",
    "Sitaram Harijan College, Nalanda",
    "Rhine Institute For Training, Ernakulam",
    "Shri Ram College of Commerce",
    "Dr. Ambedkar College, Nagpur",
    "SEEDAC College of Arts & Science SCAS, Kottappuram",
    "Himalayan School of Science & Technology HSST, Dehradun",
    "Ahmedabad Institute Of Nursing Science, Lapkaman",
    "Laxminath College, Bangawn",
    "Pendrani Mahavidylaya, Umerkote",
    "Shiva Educational Institute, Ghaziabad",
    "Asian School of Architecture & Design Innovations ASADI, Kochi",
    "Babu Shobha Ram Government Arts College, Alwar",
    "Narayana Institute Of Management, Tadipatri",
    "MISB Bocconi, Mumbai",
    "Care College of Education, Gwalior",
    "Uttamrao Patil Arts & Science College, Dahivel",
    "Government Model Degree College, Pathapatnam",
    "Unitedworld Institute of Design UID, Ahmedabad",
    "Sikkim Manipal Institute of Technology SMIT, Rangpo, East Sikkim",
    "Institute Of Engineering & Rural Technology IERT, Allahabad",
    "Shri Ram Murti Smarak Institute of Paramedical Sciences SRMSIPS, Bareilly",
    "Shri Ram Murti Smarak Institute Of Medical Sciences SRMS IMS, Bareilly",
    "SBSS College, Begusarai",
    "Pt Deen Dayal Upadhyaya Government Degree College, Saidpur",
    "Kamaxi College of Culinary Arts KCCA, Verna, Goa",
    "Symbiosis School of Sports Sciences SSSS, Pune",
    "Symbiosis Institute Of Business Management SIBM, Nagpur",
    "Symbiosis Centre for Management Studies SCMS, Bengaluru",
    "Symbiosis Centre for Management Studies SCMS, Hyderabad",
    "Symbiosis Statistical Institute SSI, Pune",
    "Symbiosis Institute of Business Management, Hyderabad SIBM-H",
    "Symbiosis Centre for International Education SCIE, Pune",
    "Symbiosis Centre for Management Studies SCMS, Nagpur",
    "Government First Grade College, Holenarasipura",
    "Vellore Institute of Technology VIT, Vellore",
    "All India Institute of Medical Sciences, Bhopal AIIMS Bhopal",
    "Shri Suresh Chandra Institute of Pharmacy, Bhiti",
    "Doon Ghati College of Professional Education, Dehradun",
    "Indian Institute of Commerce and Trade IICT, Lucknow",
    "Bitoli Devi Mahavidyalaya, Shahjahanpur",
    "Indian Institute of Public Health IIPH, Hyderabad",
    "Munna Lal & Jai Narayan Khemka Girls College, Saharanpur",
    "Basaveshwar Science College, Bagalkot",
    "Government P.G College For Women, Rohtak GPGCW, Rohtak",
    "Shaida College of Eduction, Darsi, Prakasam",
    "Shaida Moula D.El.Ed College, Darsi, Prakasam",
    "Dev Indrawati Mahavidyalaya, Ambedkar Nagar",
    "Symbiosis Centre for Distance Learning SCDL, Hyderabad",
    "LBS Centre For Science & Technology, Thiruvananthapuram",
    "Visvesvaraya National Institute of Technology",
    "DELHI UNIVERSITY",
    "Maharashtra Institute of Technology, Pune",
    "NITIE, Mumbai",
    "Institute for Excellence In Higher Education, Bhopal",
    "Great Lakes Institute of Management, Chennai",
    "KVM(MDU), Rohtak",
    "G.G.D.S.D(Punjab University), Chandigarh",
    "Keshav Mahavidhalaya(DU), Delhi",
    "RGUKT IIIT-N",
    "University Of Engineering and Management, Jaipur",
    "Villa Marie college for women, Osmania University",
    "Department of Management Studies,",
    "Indian Institute of Technology (ISM), Dhanbad",
    "Lakireddy Balireddy College of Engineering, Vijayawada",
    "GL Bajaj",
    "NDIIT",
    "GHRCE, Nagpur",
    "VPKBIET, Baramati",
    "Chandigarh Group of Colleges, Jhanjeri",
    "Kumaraguru College of Technology Business School, Coimbatore",
    "Kongu Engineering College, Perundurai",
    "NSUT",
    "IGDTUW",
    "Balaji Institute Of Modern Management, Pune",
    "Smt. Kashibai Navale College of Engineering, Pune"
]

common_skills = [
    "Communication Skills",
    "Teamwork and Collaboration",
    "Problem Solving",
    "Leadership",
    "Time Management",
    "Adaptability/Flexibility",
    "Analytical Skills",
    "Attention to Detail",
    "Creativity",
    "Interpersonal Skills",
    "Customer Service",
    "Technical Skills",
    "Microsoft Office Suite",
    "Project Management",
    "Data Analysis",
    "Research Skills",
    "Sales Skills",
    "Negotiation",
    "Presentation Skills",
    "Organizational Skills",
    "Critical Thinking",
    "Multitasking",
    "Decision Making",
    "Conflict Resolution",
    "Verbal and Written Communication",
    "Programming Languages",
    "C",
    "C++",
    "C#",
    "Java",
    "Javascript",
    "JS",
    "Python",
    "HTML",
    "CSS",
    "Node.JS",
    "React.JS",
    "Redux",
    "Mongoose",
    "MongoDB",
    "MySQL",
    "noSQL",
    "Angular",
    "Express.JS",
    "Adobe Photoshop",
    "AutoCAD",
    "Docker",
    "Word",
    "Excel",
    "PowerPoint",
    "Outlook",
]

def extract_name(resume_text):
    words = resume_text.split()
    if len(words) >= 2:
        return [' '.join(words[:2])]

def extract_email(resume_text):
    emails = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", resume_text)
    return emails

def extract_phone_numbers(resume_text):
    pattern = r'(?:(?:\+|[-])91)?([1-9][0-9]{9})\b'
    phone_numbers = re.findall(pattern, resume_text)
    return phone_numbers

def extract_dates(resume_text):
    pattern = r'\b(?:\d{4}-Present|\w+\’\d{2}\s*–\s*\w+\’\d{2}|\w+\s*’\d{2}\s*-\s*\w+\s*’\d{2}|\w+\s*’\d{2}\s*–\s*\w+\s*’\d{2}|\w+\s*\d{4}\s*–\s*present|\w+\s+\d{1,2},\s*\d{4}\s*–\s*\w+\s+\d{1,2},\s*\d{4}|\w+\s+\d{1,2},\s*\d{4}\s*–\s*\w+\s+\d{1,2},\s*\d{4}|\w+\s*\d{4}\s*–\s*\w+\s*\d{4}|\w+\s*\d{4}\s*-\s*Present|Dec\s+\d{4}\s*–\s*\w+\s+\d{4}|\w+\s*\d{4}\s*–\s*present|\w+\s+\d{4}\s*–\s*\w+\s+\d{4}|\w+\s+\d{4}\s*-\s*\w+\s+\d{4}|\w+\s+\d{4}\s*-\s*present|\w+\s+\d{4}\s*-\s*Present|\w+\s*\d{4}\s*–\s*\w+\s*\d{4}|\w+\s*-\s*\w+\s*\(?\d{4}\)?|\w+’\d{2}\s*-\s*\w+’\d{2}|\w+’\d{2}\s*-\s*Present|\w+’\d{2}\s*-\s*\w+\s*’\d{2}|\w+\s*–\s*\w+\s*’\d{2}|\w+\s*–\s*\w+\s*\d{4}|\d{1,2}/\d{4}\s*-\s*\d{1,2}/\d{4}|\d{1,2}/\d{4}\s*-\s*\d{1,2}/\d{4}|\d{1,2}/\d{4}\s*-\s*\d{1,2}/\d{4}|\d{1,2}/\d{4}\s*-\s*\d{1,2}/\d{4}|\w+\s+\d{4}\s*-\s*\w+\s+\d{4}|\w+\s*-\s*\d{4}\s*-\s*Present|\w+\s*\d{4}\s*-\s*Present|\w+\s*-\s*\d{4}\s*-\s*Present|\w+\s*\d{4}\s*-\s*\w+\s*\d{4}|\w+\s*\d{4}\s*-\s*\w+\s*\d{4}|\w+\s*\d{4}\s*-\s*\w+\s*\d{4}|\w+\s*’\d{2}\s*-\s*\w+\s*’\d{2}|\w+\s*’\d{2}\s*-\s*\w+\s*’\d{2}|\w+\s*’\d{2}\s*-\s*\w+\s*’\d{2}|\w+\s*’\d{2}\s*-\s*\w+\s*’\d{2}|\w+\s*\d{2}\s*-\s*\w+\s*\d{2}|\w+\s*\d{2}\s*-\s*\w+\s*\d{2}|\w+\s*\d{2}\s*-\s*\w+\s*\d{2}|\w+\s*\d{2}\s*-\s*\w+\s*\d{2})\b'
    dates = re.findall(pattern, resume_text)
    return dates

def calculate_months(start_date, end_date):
    current_date = datetime.datetime.now()
    if end_date.lower() in ['present', 'current']:
        end_date = f"{current_date.month}/{current_date.year}"

    try:
        start_date = datetime.datetime.strptime(start_date, "%m/%Y")
        end_date = datetime.datetime.strptime(end_date, "%m/%Y")
    except ValueError:
        try:
            start_date = datetime.datetime.strptime(start_date, "%b’%y")
            end_date = datetime.datetime.strptime(end_date, "%b’%y")
        except ValueError:
            try:
                start_date = datetime.datetime.strptime(start_date, "%Y")
                end_date = datetime.datetime.strptime(end_date, "%Y")
            except ValueError:
                return 0

    return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)

def extract_skills_from_text(resume_text):
    skills = []
    for skill in common_skills:
        if skill.lower() in resume_text.lower():
            skills.append(skill)
    return skills

def extract_university(resume_text):
    universities = []
    for university in indian_college_names:
        if university.lower() in resume_text.lower():
            universities.append(university)
    return universities

def generate_summary(resume_text):
    sentences = sent_tokenize(resume_text)

    # Generates the summary using TextRank algorithm
    summary = textrank(resume_text)

    return summary

def extract_resume_sections(resume_text):
    sections = {}

    paragraphs = [p.strip() for p in resume_text.split('\n') if p.strip()]

    section_patterns = {
        'Personal Particulars': r'PERSONAL PARTICULARS',
        'Working Experience': r'WORKING EXPERIENCE',
        'Experience': r'EXPERIENCE',
        'Achievement': r'ACHIEVEMENT:',
        'Responsibilities': r'RESPONSIBILITIES:',
        'Professional Qualification': r'PROFESSIONAL QUALIFICATION',
        'Education': r'EDUCATION',
        'Software Skills': r'SOFTWARE SKILLS',
        'Notice Period': r'NOTICE PERIOD',
        'Projects': r'PROJECTS',
        'Co-curricular activities': r'CO-CURRICULAR ACTIVITIES',
        'Extra Curricular activities': r'EXTRA CURRICULAR ACTIVITIES',
        'Skills': r'SKILLS'
    }

    current_section = None
    for paragraph in paragraphs:
        for section_name, pattern in section_patterns.items():
            if re.search(pattern, paragraph, re.IGNORECASE):
                current_section = section_name
                sections[current_section] = ""
                break
        else:
            if current_section:
                sections[current_section] += paragraph + "\n"

    for section_name in sections:
        sections[section_name] = sections[section_name].strip()

    return sections

def extract_highest_degree(resume_text):
    degrees = [
        'bachelor of technology',
        'b.tech',
        'btech',
        'bachelor of engineering',
        'master of technology',
        'mtech',
        'master of engineering',
        'bachelor of science',
        'bsc',
        'master of science',
        'msc',
        'bcom',
        'bachelor of arts',
        'b.a.',
        'master of arts',
        'post graduate',
        'bachelor of business administration',
        'mba',
        'bacholors in data science',
        'b.e',
        'b.e.',
        'masters of business administration',
        "bachelor's degree in business administration",
        'B.Tech'
    ]

    highest_degree = None
    for degree in degrees:
        if degree in resume_text.lower():
            highest_degree = degree
            break

    return highest_degree

def convert_pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text")
    return text

def get_txt_file_names(directory_path):
    txt_file_names = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.txt'):
            txt_file_names.append(file_name)
    return txt_file_names

def extract_projects(resume_text):
    projects_start_keywords = ['projects', 'project & activities', 'personal projects', 'project']
    projects_end_keywords = ['skills', 'experience', 'education', 'social impact', 'position of responsibility', 'achievements', 'extra curricular activities', 'industrial training', 'extra curriculars & achievements']

    projects_text = extract_text_between_keywords(resume_text.lower(), projects_start_keywords, projects_end_keywords)
    return projects_text if projects_text else ''

def process_resume(resume_text):
    name = extract_name(resume_text)[0] if extract_name(resume_text) else ''
    email = extract_email(resume_text)[0] if extract_email(resume_text) else ''
    phone_number = extract_phone_numbers(resume_text)[0] if extract_phone_numbers(resume_text) else ''
    sections = extract_resume_sections(resume_text)
    experience = sections.get('Experience', '')
    skills = extract_skills_from_text(resume_text)
    universities = extract_university(resume_text)
    projects = extract_projects(resume_text)
    skills_str = ', '.join(skills) if skills else ''
    university_str = ', '.join(universities) if universities else ''
    summary = generate_summary(resume_text)
    highest_degree = extract_highest_degree(resume_text)
    return name, email, phone_number, experience, skills_str, university_str, projects, summary, highest_degree

def process_resumes_in_directory(directory_path):
    txt_files = get_txt_file_names(directory_path)

    wb = Workbook()
    ws = wb.active
    ws.append(['File Name', 'Name', 'Email', 'Phone Number', 'Experience', 'Skills', 'University', 'Projects', 'Summary', 'Highest Degree'])

    for txt_file in txt_files:
        txt_file_path = os.path.join(directory_path, txt_file)

        with open(txt_file_path, 'r') as txt_file:
            resume_text = txt_file.read()

        name, email, phone_number, experience, skills_str, university_str, projects, summary, highest_degree = process_resume(resume_text)

        file_name = os.path.basename(txt_file_path)

        ws.append([file_name, name, email, phone_number, experience, skills_str, university_str, projects, summary, highest_degree])

    excel_file_path = os.path.join(directory_path, 'resume_output.xlsx')
    wb.save(excel_file_path)
    print(f"Resume data saved to {excel_file_path}")

resumes_directory = '/content/drive/MyDrive/Upload your CV (File responses)'
process_resumes_in_directory(resumes_directory)